In [1]:
import pandas as pd
import enchant   
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas import DataFrame
from openpyxl import Workbook
import xlsxwriter
import math
from collections import Counter
import os

In [2]:
def read_excel(fileName, rowCount, worksheet, ML, successListWorksheet, nameCount, failuresListWorksheet, nameCountF):
    AL = pd.read_excel(fileName)
    columns_list = AL.columns

    AL.rename(columns = {columns_list[0]:'Attendee', 
                        'Unnamed: 1':'Score','Unnamed: 2':'    1:46 PM','Unnamed: 3':'        1:55 PM','Unnamed: 4':'     2:01 PM',
                        'Unnamed: 5':'       2:41 PM'},inplace=True)
    AL= AL.drop(AL.index [  0:4  ] )
    AL= AL.drop(AL.index [  0:2  ] )
    AL.drop(AL.tail(1).index,inplace=True)
    print(AL)
    master_count=0
    for i in ML.iterrows():
        master_count=master_count+1;
    print(master_count)

    Attendance_count=0
    directmatch=0
    cosineL=[]
    cosinematch=0
    count_levenshtein_match=0
    bas_name = os.path.basename(fileName)
    split_name = bas_name.split(".")[0]
    successListWorksheet.write(nameCount, 0, split_name)
    failuresListWorksheet.write(nameCountF, 0, split_name)
    nameCount = nameCount+1
    nameCountF = nameCountF+1
    countF = 0
    for i, row in AL.iterrows():
        Attendance_count+=1
        Attendeename=f"{row['Attendee']}"
        upper_case_AN=Attendeename.upper()
        sorted_Attendeename= ''.join(sorted(upper_case_AN))
        ans=900
        flag=0
        for j, row2 in ML.iterrows():
            Mastername=f"{row2['Full Name']}"
            if(Mastername==upper_case_AN):
                flag=1
                break
            
        if(flag==1):
            directmatch=directmatch+1
        else:
            cosineL.append(Attendeename)
            count_levenshtein_match+=1
    #print(cosineL, attencount, directmatch )

    levenshtein_threshold_0=0
    levenshtein_threshold_1=0
    levenshtein_remaining=0
    for i in cosineL:
        attendeename=i
        ans=500
        upper_case_attendeename=attendeename.upper()
        upper_case_removed=upper_case_attendeename.replace(" ", "")
        sorted_attendeename= ''.join(sorted(upper_case_removed))
        for j, row2 in ML.iterrows():
            Mastername=f"{row2['Full Name']}"
            Scholar_number = f"{row2['Roll No']}"
            master_name_removed=Mastername.replace(" ", "")
            sorted_mastername= ''.join(sorted(master_name_removed))
            cos= (enchant.utils.levenshtein(str(sorted_attendeename), str(sorted_mastername)))
            if(cos<=ans):
                ans=cos
                resultant_mastername= Mastername
                resultant_scholarnumber= Scholar_number
                resultant_master_name=sorted_mastername
        print(sorted_attendeename+"  "+resultant_master_name+"  "+attendeename+"   "+resultant_mastername+"  "+str(ans))
        
        if(ans==0):
            levenshtein_threshold_0+=1
            successListWorksheet.write(nameCount, 0, resultant_mastername)
            successListWorksheet.write(nameCount, 1, resultant_scholarnumber)
            nameCount = nameCount+1
        
        elif(ans==1):
            levenshtein_threshold_1+=1
            successListWorksheet.write(nameCount, 0, resultant_mastername)
            successListWorksheet.write(nameCount, 1, resultant_scholarnumber)
            nameCount = nameCount+1
        
        
        else:
            levenshtein_remaining+=1
            failuresListWorksheet.write(nameCountF, 0, attendeename)
            countF = countF+1
            nameCountF = nameCountF+1
            
    if countF == 0:
        failuresListWorksheet.write(nameCountF, 0, "No Failures")
        nameCountF = nameCountF+1
        
    #print(cosineS1,cosineS2,cosineF)
    nameCount = nameCount+1 
    nameCountF = nameCountF+1
            
     
    master_count
    x=Attendance_count/master_count
    y=x*100
    total_per=round(y,2)
    directmatch
    total_per
    per_dir_str_match=directmatch/Attendance_count 
    per_dir_str_match=per_dir_str_match*100
    per_dir_str_match=round(per_dir_str_match,2)
    lev_str_match=count_levenshtein_match/Attendance_count
    lev_str_match=lev_str_match*100
    lev_str_match=round(lev_str_match,2)
    lev_per_thr_0=levenshtein_threshold_0/Attendance_count
    lev_per_thr_0=lev_per_thr_0*100
    lev_per_thr_0=round(lev_per_thr_0,2)
    lev_per_thr_0
    lev_per_thr_1=levenshtein_threshold_1/Attendance_count
    lev_per_thr_1=lev_per_thr_1*100
    lev_per_thr_1=round(lev_per_thr_1,2)
    lev_per_thr_rem=levenshtein_remaining/Attendance_count
    lev_per_thr_rem=lev_per_thr_rem*100
    lev_per_thr_rem=round(lev_per_thr_rem,2)
            
    worksheet.write(0,0,'Date')
    worksheet.write(0, 1, 'Total percentage of attendee')
    worksheet.write(0, 2, 'Total percentage of direct string match' )
    worksheet.write(0, 3, 'Total percentage of levenshtein match' )
    worksheet.write(0, 4, 'percentage of levenshtein match with threshold as 0' )
    worksheet.write(0, 5, 'percentage of levenshtein match with threshold as 1' )
    worksheet.write(0, 6, 'Failure percentage(percentage of remaining levenshtein match)' )
    worksheet.write(0, 7, 'Number of Attendee')
    worksheet.write(0, 8, 'Number of direct string match')
    worksheet.write(0, 9, 'Number of levenshtein string match')
    worksheet.write(0, 10, 'Number of levenshtein match-0')
    worksheet.write(0, 11, 'Number of levenshtein match-1')
    worksheet.write(0, 12,'Number of levenshtein match-remaining')
    
    date_file = fileName.split(".")
    date_tbu = date_file[0]

    worksheet.write(rowCount,0,date_tbu)
    worksheet.write(rowCount, 1, total_per)
    worksheet.write(rowCount,2, per_dir_str_match )
    worksheet.write(rowCount, 3, lev_str_match)
    worksheet.write(rowCount, 4, lev_per_thr_0)
    worksheet.write(rowCount, 5, lev_per_thr_1)
    worksheet.write(rowCount, 6, lev_per_thr_rem)
    worksheet.write(rowCount, 7, Attendance_count )
    worksheet.write(rowCount, 8, directmatch)
    worksheet.write(rowCount, 9, levenshtein_threshold_0+levenshtein_threshold_1)
    worksheet.write(rowCount, 10, levenshtein_threshold_0)
    worksheet.write(rowCount, 11, levenshtein_threshold_1)
    worksheet.write(rowCount, 12, levenshtein_remaining)
    return (nameCount, nameCountF)

In [4]:
def checkIfXlsx(fileName):
    fileData = fileName.split(".")
    fileExt = fileData[-1]
    fileBaseName = os.path.basename(fileName)
    if(fileBaseName=='levenshtein_output_set_1.xlsx' or fileBaseName=='MasterStudentList-1.xlsx' or
        fileBaseName=='Output_Success.xlsx' or fileBaseName=='Output_Failures.xlsx'or fileBaseName=='$stats_final_layer-2_final.xlsx'):
        return False
    if(fileExt=='xlsx'):
        return True
    return False

if __name__ == '__main__':
    ML = pd.read_excel("MasterStudentList-1.xlsx")
    workbook = xlsxwriter.Workbook("levenshtein_output_set_1.xlsx",)
    successListWorkbook = xlsxwriter.Workbook("Output_Success.xlsx")
    failuresListWorkbook = xlsxwriter.Workbook("Output_Failures.xlsx")
    worksheet = workbook.add_worksheet()
    successListWorksheet = successListWorkbook.add_worksheet()
    failuresListWorksheet = failuresListWorkbook.add_worksheet()
    dir_list = os.listdir()
    fileCount = 0
    nameCount = 0
    nameCountF = 0
    for file in dir_list:
        if(checkIfXlsx(fileName=file)):
            print(file)
            fileCount = fileCount+1
            (camCount, camCount_f) = read_excel(file, fileCount, worksheet, ML, successListWorksheet, nameCount, failuresListWorksheet, nameCountF)
            nameCount = camCount
            nameCountF = camCount_f
    workbook.close()
    successListWorkbook.close()
    failuresListWorkbook.close()

2021-08-16.xlsx
                                 Attendee Score     1:46 PM         1:55 PM
6                         ABHISHEK ACHARY   2/2           ✅               ✅
7                         BAMMIDI KRISHNA   2/2           ✅               ✅
8               BEESAM CHARAN KUMAR REDDY   2/2           ✅               ✅
9                 CHITRADA PRASANNA KUMAR   2/2           ✅               ✅
10                        DASARI BHARGAVI   2/2           ✅               ✅
11                  DAYANA SRI MANGA DEVI   2/2           ✅               ✅
12                          GEDALA RAKESH   2/2           ✅               ✅
13                      GURUGUBELLI SESIL   2/2           ✅               ✅
14                            IPSITA SAHU   2/2           ✅               ✅
15                         KANTALA AKHILA   2/2           ✅               ✅
16                KANUMALA VENKATA LOKESH   1/2           ✅               ❎
17                        KRISHNA BAMMIDI   2/2           ✅             

AABDHIIIKMMNRS  AABDHIIIKMMNRS  KRISHNA BAMMIDI   BAMMIDI KRISHNA  0
AAAIIIKKKLLLMRSSTUU  AAAHIIIKKLLLMRSSTUU  LAKSKMI TULASI KILARU   LAKSHMI TULASI KILARU  2
AAAADEEHHIILMNSTTX  AAAAADEEHHIILMNSTTX  SHEELADIXIT MAHANTA   SHEELA DIXIT MAHANATA  1
AAAAIKKLMNNOTUUV  AAAAIKKLMNNOTUV  VANTAKULA MOUNIKA   VANTAKULA MONIKA  1
AADDEEEIIIIKLMPSSTUUV  AADDEEEIIIKLMPSSTUUV  TEKUMUDI SIVA SAI DILEEP   TEKUMUDI SIVA SA DILEEP  1
2021-09-03.xlsx
                              Attendee Score     1:46 PM         1:55 PM  \
6              CHITRADA PRASANNA KUMAR   7/7           ✅               ✅   
7                DAYANA SRI MANGA DEVI   7/7           ✅               ✅   
8                        GEDALA RAKESH   7/7           ✅               ✅   
9                       KALAGA LIKITHA   7/7           ✅               ✅   
10                     KRISHNA BAMMIDI   3/7           ✅               ✅   
11                    LAXMI PRIYA JANI   7/7           ✅               ✅   
12      PADIGAPATI PRAVEEN KUM

AADDEEEIIIIKLMPSSTUUV  AADDEEEIIIKLMPSSTUUV  TEKUMUDI SIVA SAI DILEEP   TEKUMUDI SIVA SA DILEEP  1
AAAAAAADEEHHIIIKLMNNPRRSSSSSUVV  AAAAAAADEEEHHIIIKLMNNPRRSSSSSUVV  VERUPANDA LAKSHMI SESHASAI SRAVANI   VEERUPANDA LAKSHMI SESHA SAI SRAVANI  1
2021-09-13.xlsx
                                 Attendee Score     1:46 PM         1:55 PM  \
6                         ABHISHEK ACHARY   3/3           ✅               ✅   
7                         DASARI BHARGAVI   3/3           ✅               ✅   
8                             IPSITA SAHU   3/3           ✅               ✅   
9                          KANTALA AKHILA   3/3           ✅               ✅   
10                        KRISHNA BAMMIDI   3/3           ✅               ✅   
11                      KURUVA SURYA TEJA   3/3           ✅               ✅   
12                       LAXMI PRIYA JANI   3/3           ✅               ✅   
13                    MANUKONDA YASASWINI   3/3           ✅               ✅   
14                         MEK

AABDHIIIKMMNRS  AABDHIIIKMMNRS  KRISHNA BAMMIDI   BAMMIDI KRISHNA  0
AAAAIKKLMNNOTUUV  AAAAIKKLMNNOTUV  VANTAKULA MOUNIKA   VANTAKULA MONIKA  1
AAAABCDDEEEHKMMNRRRSUY  .AAAABCDDEEEHKMMNRRRSUY  BEESAM CHARAN KUMAR REDDY   BEESAM.CHARAN KUMAR REDDY  1
AAAADEEHHIILMNSTTX  AAAAADEEHHIILMNSTTX  SHEELADIXIT MAHANTA   SHEELA DIXIT MAHANATA  1
AAAIIIKKKLLLMRSSTUU  AAAHIIIKKLLLMRSSTUU  LAKSKMI TULASI KILARU   LAKSHMI TULASI KILARU  2
AAAAAABBHIKMRRSSTUV  AAAAAABBHIKMRRSSTUV  BASVA SAI BHARAT KUMAR   BASVA SAIBHARAT KUMAR  0
AAAAAAADHHIKLMMNNRRSSV  AAAAAAADHHIKLMMNNRRSSV  DHARMANA SRAVANA LAKSHMI   DHARMANA SRAVANALAKSHMI  0
2021-10-27.xlsx
                                 Attendee Score     1:46 PM         1:55 PM  \
6                Bandi Siva Sri Chaitanya   5/5           ✅               ✅   
7                            BENDI HARIKA   5/5           ✅               ✅   
8                     CHALAMALA GOPICHAND   4/5           ✅               ✅   
9                      CHILAMKURTHI SRIJA   5

AAAAAAADEEHHIIIKLMNNPRRSSSSSUVV  AAAAAAADEEEHHIIIKLMNNPRRSSSSSUVV  VERUPANDA LAKSHMI SESHASAI SRAVANI   VEERUPANDA LAKSHMI SESHA SAI SRAVANI  1
2021-11-03.xlsx
                                 Attendee Score     1:46 PM         1:55 PM  \
6                         ABHISHEK ACHARY   4/4           ✅               ✅   
7                Bandi Siva Sri Chaitanya   4/4           ✅               ✅   
8                     CHALAMALA GOPICHAND   4/4           ✅               ✅   
9                 CHITRADA PRASANNA KUMAR   3/4           ✅               ✅   
10                        DASARI BHARGAVI   4/4           ✅               ✅   
11                  DAYANA SRI MANGA DEVI   4/4           ✅               ✅   
12                     Jami Pranay Sundar   4/4           ✅               ✅   
13                        KRISHNA BAMMIDI   4/4           ✅               ✅   
14                       LAXMI PRIYA JANI   1/4           ✅               ❎   
15                 MALAYA KUMAR PANIGRAHI   3/4   

AABDHIIIKMMNRS  AABDHIIIKMMNRS  KRISHNA BAMMIDI   BAMMIDI KRISHNA  0
AAAADEEHHIILMNSTTX  AAAAADEEHHIILMNSTTX  SHEELADIXIT MAHANTA   SHEELA DIXIT MAHANATA  1
AAAAAAADEEHHIIIKLMNNPRRSSSSSUVV  AAAAAAADEEEHHIIIKLMNNPRRSSSSSUVV  VERUPANDA LAKSHMI SESHASAI SRAVANI   VEERUPANDA LAKSHMI SESHA SAI SRAVANI  1
AAAAAABBHIKMRRSSTUV  AAAAAABBHIKMRRSSTUV  BASVA SAI BHARAT KUMAR   BASVA SAIBHARAT KUMAR  0
AAAABCDDEEEHKMMNRRRSUY  .AAAABCDDEEEHKMMNRRRSUY  BEESAM CHARAN KUMAR REDDY   BEESAM.CHARAN KUMAR REDDY  1
AAAAAAADHHIKLMMNNRRSSV  AAAAAAADHHIKLMMNNRRSSV  DHARMANA SRAVANA LAKSHMI   DHARMANA SRAVANALAKSHMI  0
AAAIIIKKKLLLMRSSTUU  AAAHIIIKKLLLMRSSTUU  LAKSKMI TULASI KILARU   LAKSHMI TULASI KILARU  2
ADEIIIJNOST  ADEIIIJNOST  NETAJI ODISI   ODISI NETAJI  0
AADDEEEIIIIKLMPSSTUUV  AADDEEEIIIKLMPSSTUUV  TEKUMUDI SIVA SAI DILEEP   TEKUMUDI SIVA SA DILEEP  1
AAAAIKKLMNNOTUUV  AAAAIKKLMNNOTUV  VANTAKULA MOUNIKA   VANTAKULA MONIKA  1
2021-11-22-2.xlsx
                                 Attendee Score     1:46 PM